# Processing of TopAGNPS data for CCHE1D

## Summary

### Inputs
The `topagnps2cche1d` module is a set of Python functions that process some of the outputs of TopAGNPS i.e.:
- `AnnAGNPS_Reach_IDs.asc`
    - The raster file of the different reaches obtained from processing a DEM file
- `AgFlow_Reach_Data.csv`
    - csv file containing information about each reach, the position of the upstream and downstream row/column coordinates of each reach, the receiving reach etc.
- `FLOVEC.asc`
    - The flow direction raster file (used to determine for each junction which inflow is on the right and which one comes from the left)
- `NETW.asc`
    - Raster file indicating what is the Strahler number of each pixel of each reach

### Outputs
The module produces the:
- `*_nodes.dat`
- `*_channel.dat`
- `*_link.dat`
- `*_reach.dat`
- `*_csec.dat`
- `*_csprf.dat`

files that can be used as inputs by CCHE1D

### Assumptions made
- This tool assumes that every junction consists strictly of two upstream inflows. This is insured by TopAGNPS.
- For now, every node is assigned a default cross-section

### Additional tools
Functions to read, process and visualize `.asc` raster files as well as CCHE1D `*_nodes.dat` files are provided in the module. Functions to convert row/col to latitude/longitude coordinates and inversely are provided making use of the `osgeo.gdal` and `affine` modules.


### Loading the module

In [1]:
import topagnps2cche1d.tools as t2c
import plotly.express as px

### Files location

In [2]:
# Main Folder
# srcfolder = '../input_data/goodwin_creek_files/'
srcfolder = '../input_data/goodwin_creek_1m/'

# File names
filepath_annagnps_reach_ids = srcfolder + 'AnnAGNPS_Reach_IDs.asc'
filepath_agflow = srcfolder + 'AgFlow_Reach_Data.csv'
filepath_flovec = srcfolder + 'FLOVEC.asc'
filepath_netw = srcfolder + 'NETW.asc'
filepath_dednm = srcfolder + 'RELIEF.asc'

visualize = False

### Reading the reach data
- `img` is a Numpy array that represents the raster of the `AnnAGNPS_Reach_IDs.asc` file
- `geoMatrix` is a 6-tuple holding the coefficients of the affine transformation to convert row and columns to latitude and longitude
- `ncols` and `nrows` are the dimensions of the raster
- `nodataval` is the value used in the raster to represent the absence of data
- `dataset` is a `gdal` object that holds all the data related to the raster 

In [3]:
img, geoMatrix, ncols, nrows, nodataval, dataset = t2c.read_esri_asc_file(filepath_annagnps_reach_ids)

### Building of the channel network

The channel network is constructed for CCHE1D using the connectivity data in the `Agflow_Reach_Data.csv` file

In [4]:
dfagflow = t2c.read_agflow_reach_data(filepath_agflow)
dfagflow.head(n=10)

,Reach_ID,Upstream_End_Row,Upstream_End_Column,Downstream_End_Row,Downstream_End_Column,Receiving_Reach,Drainage_Area_[ha],Average_Elevation_[m],Reach_Length_[m],Distance_Upstream_End_to_Outlet_[m],Distance_Downstream_End_to_Outlet_[m],Reach_Slope_[m/m],Contributing_Cell_ID_Source,Contributing_Cell_ID_Right,Contributing_Cell_ID_Left
0,1,6048,578,6048,578,1,2135.57,67.45,0.00,0.00,0.00,0.00283,0,0,0
1,2,5978,761,6048,578,1,2135.57,67.45,247.14,247.14,0.00,0.00283,0,22,23
2,3,5947,822,5978,762,2,2128.12,67.82,75.01,322.15,247.14,0.00134,0,32,33
3,4,5861,859,5946,823,3,2103.89,67.94,103.91,426.06,322.15,0.00097,0,42,43
4,5,5556,1051,5860,859,4,2101.50,68.30,406.73,832.79,426.06,0.00172,0,52,53
5,6,5344,1058,5555,1051,5,59.98,70.49,242.20,1074.98,832.79,0.01032,0,62,63
6,7,5339,1020,5343,1057,6,14.00,71.40,44.56,1119.54,1074.98,0.01571,0,72,73
7,8,5378,970,5340,1019,7,5.97,72.86,83.33,1202.86,1119.54,0.01681,0,82,83
8,9,5303,709,5379,969,8,2.65,75.08,335.20,1538.06,1202.86,0.01343,91,92,93
9,10,5370,898,5378,969,8,2.46,73.62,81.70,1284.56,1202.86,0.00734,101,102,103


### Visualization of Strahler Number for the network

The following diagram illustrates the Strahler number for a river network and is an indication of the network complexity

<img src='content/strahler.png' align="center" alt=tree-traversal width='500'/>



In [5]:
img_strahler, geoMatrix, _, _, _, _, = t2c.read_esri_asc_file(filepath_netw)

if visualize:
    fig = t2c.visualize_strahler_number(img_strahler, geoMatrix, renderer='browser')

### Generating the CCHE1D files

All the heavy lifting is done inside the `convert_topagnps_output_to_cche1d_input` and `create_cche1d_tables` functions.

The outputs are the Pandas DataFrame of the nodes, channel, and link files. `img_reach` is a Numpy array with the reordered reach numbering

#### Generating a default cross-section for all nodes of the network

CP_W and CP_Z are the x and y coordinates of the cross-section and CP_RGH the Manning's roughness coefficient

In [6]:
default_xsection = {'type' : 'default',
                    'CP_Ws': [-43, -35, -13, -10, 10, 13, 35, 43],
                    'CP_Zs': [6, 2, 2, 0, 0, 2, 2, 6],
                    'CP_RGHs': [0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025]}

fig = px.line(x = default_xsection['CP_Ws'], y = default_xsection['CP_Zs'], markers='o', title = 'Default Cross-Section')
fig.update_layout(xaxis_title='x (m)', yaxis_title='y (m)')
fig.show()

In [26]:
distance = 10
min_strahler = 3
# df_nodes, df_channel, df_link, df_reach, df_csec, df_csprf, img_reach = t2c.convert_topagnps_output_to_cche1d_input(filepath_agflow, filepath_flovec, filepath_annagnps_reach_ids, filepath_netw, default_xsection, min_strahler, distance)
df_nodes, df_channel, df_link, df_reach, df_csec, df_csprf, img_reach, cche1d_to_annagnps_reaches = t2c.convert_topagnps_output_to_cche1d_input(filepath_agflow, filepath_annagnps_reach_ids, filepath_netw, default_xsection, min_strahler, distance)

#### Reordered reaches
Note that the reach containing the outlet is the reach with the highest reach number

In [27]:
if visualize:
    fig = t2c.visualize_reaches_id(img_reach, geoMatrix, title='New Reach IDs', renderer='browser')

In [28]:
reaches_cche1d_topagnps = t2c.dict_cche1d_annagnps_to_df(cche1d_to_annagnps_reaches)
reaches_cche1d_topagnps

,CCHE1D_Reach,TopAGNPS_Reach
0,26,207
1,25,224
2,27,190
3,23,300
4,21,392
5,22,353
6,28,181
7,24,269
8,29,168
9,30,140


In [29]:
reaches_cche1d_topagnps.to_csv(f'../input_data/goodwin_creek_1m/cche1d/cche1d_annagnps_reach_equivalent_min_strahler_{min_strahler}_step_{distance}m.csv', index=False)


### Assemble cell, reach, outflow node file

#### Visualization of the nodes as seen by CCHE1D:

| ND_TYPE | Meaning |
|---------|---------|
| 0       | Source node |
| 2       | *Upstream* junction node |
| 3       | *Downstream/Receiving* junction node |
| 6       | Normal node |
| 9       | Outlet node |

Note: The coordinates of points 2 and 3 are the exact same for any given junction


In [30]:
visualize = False
if visualize:
    fig = t2c.visualize_cche1d_nodes(df_nodes, renderer='browser')

### Writing of the files

In [31]:
outputfolder = '../input_data/goodwin_creek_1m/cche1d/' # Location to write the files
casename = 'goodwin_creek' # Name of the case for these files

In [32]:
t2c.write_cche1d_dat_file(f'{outputfolder}{casename}_min_strahler_{min_strahler}_step_{distance}m', df_nodes)
t2c.write_cche1d_dat_file(f'{outputfolder}{casename}_min_strahler_{min_strahler}_step_{distance}m', df_channel)
t2c.write_cche1d_dat_file(f'{outputfolder}{casename}_min_strahler_{min_strahler}_step_{distance}m', df_link)
t2c.write_cche1d_dat_file(f'{outputfolder}{casename}_min_strahler_{min_strahler}_step_{distance}m', df_reach)
t2c.write_cche1d_dat_file(f'{outputfolder}{casename}_min_strahler_{min_strahler}_step_{distance}m', df_csec)
t2c.write_cche1d_dat_file(f'{outputfolder}{casename}_min_strahler_{min_strahler}_step_{distance}m', df_csprf)

../input_data/goodwin_creek_1m/cche1d/goodwin_creek_min_strahler_3_step_10m_nodes.dat successfully written
../input_data/goodwin_creek_1m/cche1d/goodwin_creek_min_strahler_3_step_10m_channel.dat successfully written
../input_data/goodwin_creek_1m/cche1d/goodwin_creek_min_strahler_3_step_10m_link.dat successfully written
../input_data/goodwin_creek_1m/cche1d/goodwin_creek_min_strahler_3_step_10m_reach.dat successfully written
../input_data/goodwin_creek_1m/cche1d/goodwin_creek_min_strahler_3_step_10m_csec.dat successfully written
../input_data/goodwin_creek_1m/cche1d/goodwin_creek_min_strahler_3_step_10m_csprf.dat successfully written
